In [1]:
using Revise
using PauliPropagation

In [2]:
nq = 8

8

In [3]:
symbs = [:I for _ in 1:nq]
symbs[round(Int, nq/2)] = :Z   # as symbol. Also works but is slower.

obsint = symboltoint(symbs)  # for performance we work with bitoperations
show(obsint, nq)

IIIZIIII


In [4]:
nl = 4
topo = bricklayertopology(nq; periodic=false)
# topo = get2dtopology(4, 4)
circ = hardwareefficientcircuit(nq, nl; topology=topo)
# circ = efficientsu2circuit(nq, nl; topology=topo)
fastcirc = tofastgates(circ)
m = length(fastcirc)

124

In [5]:
using Random
Random.seed!(42)
thetas = randn(m);

In [6]:
W = Inf;                   # maximal operator weight.
min_abs_coeff = 0;          # neglect small coefficients. Only for numerical and hybrid PP.

#### Numerical Pauli Propagation
Propagates numerical coefficients.

In [7]:
@time dnum = mergingbfs(fastcirc, obsint, thetas; max_weight=W, min_abs_coeff=min_abs_coeff);
@show length(dnum)   # number of unique Pauli ops
overlapwithzero(dnum) # expectation

  0.624414 seconds (678.06 k allocations: 48.279 MiB, 82.52% compilation time)
length(dnum) = 53247


0.21720058439757214

In [8]:
show(dnum)

Dict{UInt16, Float64} with 53247 entries:
  XZIIXXZX => 9.342063615803493e-7
  IIIXIIZY => 9.36513557295548e-6
  YZZIYXZX => 3.818927407510158e-6
  ZXYYYXII => -0.0001441502376292454
  IZYZIIXY => -1.2333560289051895e-6
  XXIIYZXI => 2.2079024544367792e-7
  IZZXIZYZ => 1.4947093598678283e-7
  YYXIYYYZ => 3.270505758837586e-7
  YZIXXZII => -0.0035533574412631294
  IXIYIIZX => -5.727900984982438e-7
  IYZZZYYX => -3.3665247394632922e-6
  YIYZYIZZ => -8.123208720289147e-9
  XIZZXXZZ => 4.2359204503716784e-8
  XZZXZZYZ => -2.3431691700451155e-8
  ZZXZZXII => -0.00012063886116623598
  XZYXIYYZ => -4.8324363250977076e-8
  XXXYIYXZ => -3.3641267519710954e-7
  IIIXZZXZ => 9.209644347437583e-7
  ZZYXYYYY => 5.995964204493712e-7
  XXXXZZXY => -2.2673757070594606e-7
  ⋮


#### Hybrid Pauli Propagation
Propagates numerical coefficients, but can but truncated like the surrogate.

In [9]:
max_freq = Inf   # max frequency, i.e., max number of sines and cosines per path

Inf

In [10]:
@time dhyb = mergingbfs(fastcirc, obsint, NumericPathProperties(1.0), thetas; max_weight=W, max_freq=Inf, min_abs_coeff=min_abs_coeff);
@show length(dhyb)
overlapwithzero(dhyb)

  0.442694 seconds (966.14 k allocations: 53.718 MiB, 5.56% gc time, 65.29% compilation time)
length(dhyb) = 53247


0.21720058439757214

In [11]:
show(dhyb)

Dict{UInt16, NumericPathProperties} with 53247 entries:
  XZIIXXZX => NumericPathProperties(9.342063615803493e-7, nsins=18, ncos=9, freq=40)
  IIIXIIZY => NumericPathProperties(9.36513557295548e-6, nsins=11, ncos=5, freq=29)
  YZZIYXZX => NumericPathProperties(3.818927407510158e-6, nsins=16, ncos=8, freq=42)
  ZXYYYXII => NumericPathProperties(-0.0001441502376292454, nsins=13, ncos=8, freq=31)
  IZYZIIXY => NumericPathProperties(-1.2333560289051895e-6, nsins=16, ncos=8, freq=36)
  XXIIYZXI => NumericPathProperties(2.2079024544367792e-7, nsins=17, ncos=9, freq=35)
  IZZXIZYZ => NumericPathProperties(1.4947093598678283e-7, nsins=15, ncos=9, freq=37)
  YYXIYYYZ => NumericPathProperties(3.270505758837586e-7, nsins=19, ncos=11, freq=42)
  YZIXXZII => NumericPathProperties(-0.0035533574412631294, nsins=11, ncos=6, freq=31)
  IXIYIIZX => NumericPathProperties(-5.727900984982438e-7, nsins=19, ncos=11, freq=39)
  IYZZZYYX => NumericPathProperties(-3.3665247394632922e-6, nsins=13, ncos=7, freq=3

#### Pauli Propagation Surrogate
Builds a graph that can later be evaluated.

In [12]:
@time dsym = mergingbfs(circ, operatortopathdict(obsint), zeros(m); max_weight=W, max_freq=max_freq);
@show length(dsym)

final_nodes = collect(pth.coeff for (obs, pth) in zerofilter(dsym));
final_eval_node = PauliGateNode(parents=final_nodes, trig_inds=zeros(Int, length(final_nodes)), signs=ones(length(final_nodes)), param_idx=1, cummulative_value=0.0);
resetnodes(final_eval_node)
resetnodes(final_eval_node)
@time eval_list = gettraceevalorder(final_eval_node, zeros(m));
length(eval_list)  # The list of all nodes. The order is such that one can savely be evaluated after the other.

  1.488194 seconds (11.51 M allocations: 621.602 MiB, 19.07% gc time, 31.54% compilation time)
length(dsym) = 53247
  0.126804 seconds (32.10 k allocations: 5.161 MiB, 49.56% compilation time)


225500

In [13]:
@time expectation(eval_list, thetas)    # This is actually not always faster than numerical propagation, but in interesting cases it is by a lot.
                                        # making this always at least as fast is work in progress. Graph traversal is hard.

  0.106451 seconds (62.00 k allocations: 4.309 MiB, 55.69% compilation time)


0.2172005843975723